In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_lattice as tfl
from concurrent import futures

import itertools

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

importing Jupyter notebook from common.ipynb


In [3]:
dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset, 54)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

Rejected 6196563 points (56.849770%)


In [4]:
preprocessed_dataset = dataset.copy()
col = preprocessed_dataset.first_val
preprocessed_dataset.first_val = np.log(col + 1)
# for col_name in preprocessed_dataset.columns[:-1]:
#     col = preprocessed_dataset[col_name]
#     preprocessed_dataset[col_name] = (col - col.mean())/col.std()

In [5]:
preprocessed_dataset

,row,rising_idx,falling_idx,first_val,last_val,delay
0,0,131,315,6.444131,0,694.991
1,1,91,263,6.483107,0,1061.620
2,2,110,284,6.163315,0,879.057
3,3,155,338,5.993961,0,512.428
4,4,175,362,6.142037,0,328.362
...,...,...,...,...,...,...
10899831,49,131,310,3.044522,0,708.468
10899832,50,173,358,3.044522,0,342.287
10899833,51,343,562,2.639057,0,-940.805
10899834,52,487,739,2.302585,0,-1857.720


In [6]:
kp_initializers = {
    'row': "quantile", 'rising_idx': "quantile", 'falling_idx': "quantile", 'first_val': "uniform", 'last_val': "uniform"
}
monotonicities = {
    "rising_idx": -1,
    "falling_idx": -1
}

In [7]:
keras.backend.clear_session()

In [69]:
def trainLattice(model, cols, epochs, log_path):
    features = [preprocessed_dataset[col].values for col in cols]
    target = preprocessed_dataset["delay"]
    csv_logger = keras.callbacks.CSVLogger(log_path, append=True, separator=';')
    early_stop = keras.callbacks.EarlyStopping(
        monitor='loss', patience=8, mode='min', restore_best_weights=True)
    

    history = model.fit(features,
            target,
            batch_size=256,
            epochs=epochs,
            validation_split=0.2,
            shuffle=False, workers=32, use_multiprocessing=True, callbacks=[csv_logger, early_stop], verbose=2)
    return history

In [67]:
def buildLattice(num_keypoints, lattice_size, columns, num_lattices_per_layer, depth):
    base_inputs = [keras.layers.Input(shape=[1]) for _ in columns]
    for dp in range(depth):
        if dp == 0:
            tmp = []
            # handle initial layer
            for inpt, ft in zip(base_inputs, columns):
                if ft != "row":
                    if kp_initializers[ft] == "quantile":
                        quantile_vals = [i/(num_keypoints - 1.0) for i in range(num_keypoints)]
                        keypoints = dataset[ft].quantile(quantile_vals).values
                    if kp_initializers[ft] == "uniform":
                        keypoints = np.linspace(preprocessed_dataset[ft].min(), preprocessed_dataset[ft].max(), num=num_keypoints)

                    calibrator = tfl.layers.PWLCalibration(
                        input_keypoints=keypoints, dtype=tf.float32, output_min=0.0, output_max=lattice_size - 1.0,
                        units = num_lattices_per_layer,
                        monotonicity=monotonicities.get(ft, 0)
                    )(inpt)
                else:
                    # row is categorical
                    calibrator = tfl.layers.CategoricalCalibration(
                        num_buckets=preprocessed_dataset[ft].nunique(),
                        output_min = 0.0,
                        output_max = lattice_size - 1.0,
                        units = num_lattices_per_layer
                    )(inpt)
                tmp.append(calibrator)
            if num_lattices_per_layer != 1:
                combined_calibrators = keras.backend.stack(tmp, axis=2)
            else:
                combined_calibrators = keras.layers.concatenate(tmp)
            num_inputs = len(base_inputs)
        else:
            tmp = tfl.layers.PWLCalibration(input_keypoints=np.linspace(0, 1, num=num_keypoints), output_min=0.0,
                                                             output_max=lattice_size - 1.0,
                                                             units = num_lattices_per_layer)(inputs)
            # now need to stack copies of tmp
            combined_calibrators = keras.backend.stack([tmp]*num_lattices_per_layer, axis=1)
            num_inputs = num_lattices_per_layer
        
        # construct a multi-unit lattice
        print("Combined Calibrators:", combined_calibrators.shape)
        inputs = tfl.layers.Lattice(
            lattice_sizes=[lattice_size for _ in range(num_inputs)], output_min = 0.0, output_max = 1.0,
            units = num_lattices_per_layer
        )(combined_calibrators)
        print("Inputs:", inputs.shape)

    if num_lattices_per_layer != 1:
        inputs = tfl.layers.Linear(
            num_input_dims=num_lattices_per_layer, normalization_order=1,
            kernel_regularizer=keras.regularizers.l1(0.01))(inputs)
    calibrated_output = tfl.layers.PWLCalibration(
            input_keypoints=np.linspace(0, 1, num=num_keypoints), output_min=preprocessed_dataset['delay'].min(),
            output_max=preprocessed_dataset['delay'].max(), monotonicity="increasing")(inputs)
    model = keras.models.Model(inputs=base_inputs, outputs=calibrated_output)
    model.compile(loss=keras.losses.mean_absolute_error,
                optimizer=keras.optimizers.Adam(), metrics=["mse"])
    return model


In [ ]:
keras.backend.clear_session()

def train_model(lattices_per_layer, depth, load_model=True):
    model_path = f'./Hypercube_8_2_{lattices_per_layer}_{depth}'
    if load_model:
        lattice = keras.models.load_model(model_path+".h5", custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    else:
        lattice = buildLattice(8, 2, columns[:-1], lattices_per_layer, depth)
    lattice.compile(loss=keras.losses.mean_absolute_error,
            optimizer=keras.optimizers.Adagrad(), metrics=["mse"])
    history = trainLattice(lattice,columns[:-1], 16, model_path + ".csv")
    lattice.save(model_path + ".h5")



while reruns:
    with futures.ThreadPoolExecutor() as pool:
    #     combinations = itertools.product([1, 2, 4, 6, 8], [1, 2, 3, 4, 5, 6, 7, 8])
        combinations = reruns
        for args in combinations:
            print("Training:", *args)
            pool.submit(train_model, *args)
#             train_model(*args)
    break

Training: 1 1
Training: 2 1
Training: 2 2
Training: 2 3
Training: 2 4
Training: 2 6
Training: 2 8
Training: 4 3
Training: 4 4
Training: 4 5
Training: 4 6
Training: 4 7
Training: 4 8
Training: 6 1
Training: 6 2
Training: 6 4
Training: 6 5
Training: 6 7
Training: 6 8
Training: 8 1
Training: 8 2
Training: 8 3
Training: 8 4
Training: 8 5
Training: 8 6
Training: 8 7
Training: 8 8
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
Train on 3762662 samples, validate on 940666 s

In [61]:
reruns = []
files = !ls *.csv
for fn in files:
    with open(fn) as f:
        for line in f:
            pass
        split = fn.split("_")
        lpl = int(split[-2])
        depth = int(split[-1].split(".")[0])
        reruns.append((lpl, depth))

In [62]:
reruns

[(1, 1),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 6),
 (2, 8),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (6, 1),
 (6, 2),
 (6, 4),
 (6, 5),
 (6, 7),
 (6, 8),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8)]

In [59]:
train_model(6, 7, True)

Train on 3762662 samples, validate on 940666 samples
Epoch 1/16
3762662/3762662 - 364s - loss: 5.4337 - mse: 331.3935 - val_loss: 5.2917 - val_mse: 334.0094
Epoch 2/16
3762662/3762662 - 352s - loss: 5.2474 - mse: 325.6170 - val_loss: 5.2407 - val_mse: 332.0394
Epoch 3/16
3762662/3762662 - 358s - loss: 5.2113 - mse: 323.2291 - val_loss: 5.2123 - val_mse: 328.4985
Epoch 4/16
3762662/3762662 - 361s - loss: 5.1882 - mse: 320.7349 - val_loss: 5.1933 - val_mse: 325.7516
Epoch 5/16
3762662/3762662 - 354s - loss: 5.1718 - mse: 320.0960 - val_loss: 5.1760 - val_mse: 324.9814
Epoch 6/16
3762662/3762662 - 356s - loss: 5.1586 - mse: 319.3404 - val_loss: 5.1629 - val_mse: 323.9946
Epoch 7/16
3762662/3762662 - 356s - loss: 5.1475 - mse: 318.7658 - val_loss: 5.1510 - val_mse: 324.1135
Epoch 8/16
3762662/3762662 - 360s - loss: 5.1381 - mse: 318.1583 - val_loss: 5.1400 - val_mse: 323.0582
Epoch 9/16
3762662/3762662 - 367s - loss: 5.1291 - mse: 317.7024 - val_loss: 5.1294 - val_mse: 322.7847
Epoch 10/16